In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.ar_model import AR

# Time Series Regressions

In [2]:
roto = pd.read_csv('http://rotoguru1.com/cgi-bin/nba-dhd-2017.pl?&user=jasonjjchen&key=J8987209841',sep = ':')

/Users/royh21k/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
roto1 = roto.iloc[:-1,:]
roto1.loc[:,'Date'] = pd.to_datetime(roto1.Date.astype(int),format='%Y%m%d')
# roto1 = roto1.iloc[2:,:] #remove na for unfinished games
rotocols = roto1.columns[[1,2,3,4,5,6,8,9,10,11,12,16,24,25,31]]
roto1 = roto1[rotocols] #clean columns
namechange = lambda x: x.replace('Derrick Jones', 'Derrick Jones, Jr.').replace('James Ennis', \
            'James Ennis III').replace('Guillermo Hernangomez', 'Willy Hernangomez').replace('Joseph Young', \
            'Joe Young').replace('Timothe Luwawu', 'Timothe Luwawu-Cabarrot').replace('Walter Tavares', \
            'Edy Tavares').replace('John Lucas', 'John Lucas III').replace('T.J. Warren', \
            'TJ Warren').replace('DeAndre Bembry', 'DeAndre\' Bembry').replace('P.J. Tucker', \
            'PJ Tucker').replace('Larry Nance', 'Larry Nance Jr.').replace('C.J. Wilcox', \
            'CJ Wilcox').replace('C.J. McCollum', 'CJ McCollum').replace('Wes Matthews', \
            'Wesley Matthews').replace('J.J. Redick', 'JJ Redick').replace('K.J. McDaniels', \
            'KJ McDaniels').replace('A.J. Hammons', 'AJ Hammons').replace('Louis Williams', \
            'Lou Williams').replace('Wade Baldwin', 'Wade Baldwin IV').replace('Kelly Oubre', \
            'Kelly Oubre Jr.').replace('Johnny O\'Bryant', 'Johnny O\'Bryant III').replace('J.R. Smith', \
            'JR Smith').replace('C.J. Miles', 'CJ Miles').replace('R.J. Hunter', 'RJ Hunter').replace('Otto Porter', \
            'Otto Porter Jr.').replace('Jose Barea', 'J.J. Barea').replace('Ishmael Smith', \
            'Ish Smith').replace('Nene Hilario', 'Nene').replace('Maurice N\'dour', 'Maurice Ndour')
#function to standardize team abbreviations
teamAbbrchange = lambda x: x.replace('atl', u'ATL').replace('bkn', u'BKN').replace('bos', u'BOS').replace('cha', \
            u'CHA').replace('chi', u'CHI').replace('cle', u'CLE').replace('dal', u'DAL').replace('den', \
            u'DEN').replace('det', u'DET').replace('gsw', u'GSW').replace('hou', u'HOU').replace('ind',\
            u'IND').replace('lac', u'LAC').replace('lal', u'LAL').replace('mem', u'MEM').replace('mia', \
            u'MIA').replace('mil', u'MIL').replace('min', u'MIN').replace('nor', u'NOP').replace('nyk', \
            u'NYK').replace('okc', u'OKC').replace('orl', u'ORL').replace('phi', u'PHI').replace('pho', \
            u'PHX').replace('por', u'POR').replace('sac', u'SAC').replace('sas', u'SAS').replace('tor', \
            u'TOR').replace('uta', u'UTA').replace('was', u'WAS')

/Users/royh21k/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [4]:
roto1['First  Last'] = roto1['First  Last'].apply(namechange)

In [5]:
#clean up draft kings data
dkgamelog = roto1[-roto1['DK pos'].isnull()]
dkgamelog = dkgamelog[-dkgamelog['Start'].isnull()]
dkgamelog = dkgamelog.rename(columns = {'First  Last': 'PLAYER', 'Date': 'GAME_DATE'})
dkgamelog['Start'] = dkgamelog['Start'].astype(int)
temp = dkgamelog['DK pos'].astype(int).astype(str).str.join('@').str.get_dummies('@')
temp.columns = 'POSITION:' + temp.columns
# temp.columns = ['PG', 'SG', 'SF', 'PF', 'C']
dkgamelog['DK pos'] = dkgamelog['DK pos'].astype(int).astype(str).str.join('@').str.split('@')
dkgamelog = pd.concat([dkgamelog[['PLAYER', 'GAME_DATE', 'DK Sal', 'DK Change', 'DKP', 'Start', 'DK pos']], temp], axis= 1)
dkgamelog.loc[dkgamelog['DK Change'].isnull(), 'DK Change'] = 0

In [6]:
roto1.head()

Last, First   First  Last       Date Team  Opp H/A  GTime(ET)  \
0  Williams, Louis  Lou Williams 2017-03-28  hou  gsw   H       20.0   
1  Williams, Louis  Lou Williams 2017-03-26  hou  okc   H       15.5   
2  Williams, Louis  Lou Williams 2017-03-24  hou  nor   H       20.0   
3  Williams, Louis  Lou Williams 2017-03-20  hou  den   H       20.0   
4  Williams, Louis  Lou Williams 2017-03-18  hou  den   A       21.5   

   Team pts  Opp pts  Start    Minutes    DKP  DK Sal  DK Change  DK pos  
0       NaN      NaN    NaN        NaN    NaN  5200.0      400.0    12.0  
1     137.0    125.0    0.0  30.966667  44.75  4800.0     -300.0    12.0  
2     117.0    107.0    0.0  19.550000  10.00  5100.0     -400.0    12.0  
3     125.0    124.0    0.0  16.866667  13.25  5500.0     -300.0    12.0  
4     109.0    105.0    0.0  22.750000  20.75  5800.0        0.0    12.0

In [7]:
TS = roto1[['First  Last', 'DK pos', 'Date']]
TS.head()

First  Last  DK pos       Date
0  Lou Williams    12.0 2017-03-28
1  Lou Williams    12.0 2017-03-26
2  Lou Williams    12.0 2017-03-24
3  Lou Williams    12.0 2017-03-20
4  Lou Williams    12.0 2017-03-18

In [8]:
#Filter out all players causing exceptions and players with very little observations.
roto1 = roto1.dropna()
exception_player = []
AR_predictions = pd.DataFrame([])
for name,group in roto1.groupby('First  Last'):
    print 'Processing: ', name
    TS = group.iloc[::-1]
    rng = pd.date_range('1/1/2000', periods = len(TS), freq = 'D')
    TS['Date_New'] = rng
    TS = TS.set_index('Date_New')
    TS = TS['DKP']
    #build the regression model
    try:
        model = AR(TS)
        model_fit = model.fit()
        predicted_value = model_fit.predict(start = len(TS) - 1, end = len(TS) - 1, dynamic = False)
    except Exception as e:
        print e.args
        exception_player.append(name)
    if len(group) <= 20 and name not in exception_player:
        exception_player.append(name)
    if len(group['DK pos'].unique()) == 1 and group['DK pos'].unique()[0] == 0 and name not in exception_player:
        exception_player.append(name)

Processing:  AJ Hammons
Processing:  Aaron Brooks
Processing:  Aaron Gordon
Processing:  Aaron Harrison


/Users/royh21k/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/royh21k/anaconda/lib/python2.7/site-packages/statsmodels/base/data.py:503: FutureWarning: TimeSeries is deprecated. Please use Series
  return TimeSeries(result, index=self.predict_dates)


Processing:  Adreian Payne
Processing:  Al Horford
Processing:  Al Jefferson
Processing:  Al-Farouq Aminu
Processing:  Alan Anderson
Processing:  Alan Williams
Processing:  Alec Burks
Processing:  Alex Abrines
Processing:  Alex Len
Processing:  Alexis Ajinca
Processing:  Allen Crabbe
Processing:  Alonzo Gee
Processing:  Amir Johnson
Processing:  Anderson Varejao
Processing:  Andre Drummond
Processing:  Andre Iguodala
Processing:  Andre Roberson
Processing:  Andrew Bogut
Processing:  Andrew Harrison
Processing:  Andrew Nicholson
Processing:  Andrew Wiggins
Processing:  Anthony Bennett
Processing:  Anthony Brown
Processing:  Anthony Davis
Processing:  Anthony Morrow
Processing:  Anthony Tolliver
Processing:  Archie Goodwin
Processing:  Arinze Onuaku
Processing:  Aron Baynes
Processing:  Arron Afflalo
Processing:  Austin Rivers
Processing:  Avery Bradley
Processing:  Axel Toupane
('maxlag should be < nobs',)
Processing:  Ben Bentil
('maxlag should be < nobs',)
Processing:  Ben McLemore
Pr

In [9]:
print len(exception_player)
exception_player[0]

30


'Anthony Brown'

In [10]:
#get injury list to filter out injured players from draft kings data
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
driver = webdriver.Chrome()
driver.get('http://www.cbssports.com/nba/injuries/daily')
time.sleep(2)
plyr = driver.find_elements_by_xpath('//*[@id="DailyTableData"]/tr')
injurylist = {'DATE': [], 'POS': [], 'PLAYER': [], 'TEAM': [], 'INJURY': [], 'EXPECTED_RETURN': []}
for ply in plyr:
    injurylist['DATE'].append(ply.find_element_by_xpath('.//td[1]/div').text)
    injurylist['POS'].append(ply.find_element_by_xpath('.//td[2]/div').text)
    injurylist['PLAYER'].append(ply.find_element_by_xpath('.//td[3]/div').text)
    injurylist['TEAM'].append(ply.find_element_by_xpath('.//td[4]/div').text)
    injurylist['INJURY'].append(ply.find_element_by_xpath('.//td[5]/div').text)
    injurylist['EXPECTED_RETURN'].append(ply.find_element_by_xpath('.//td[6]/div').text)
injurylist = pd.DataFrame(injurylist)
injurylist['PLAYER'] = injurylist['PLAYER'].apply(namechange)
driver.close()

In [11]:
print len(injurylist.PLAYER.tolist())
injurylist.PLAYER.tolist()[0]

69


u'Arron Afflalo'

In [12]:
#Combine the fitlering 
exception_player_p1 = [x for x in exception_player if x not in injurylist.PLAYER.tolist()]
exception_player_p2 = [x for x in injurylist.PLAYER.tolist() if x not in exception_player]
unique_exception_player = exception_player_p1 + exception_player_p2
print len(unique_exception_player)
#unique_exception_player

99


In [13]:
#Regression loops
roto1 = roto1.dropna()
#exception_player = []
AR_predictions = pd.DataFrame([])
last_game_dates = []
for name,group in roto1.groupby('First  Last'):
    if name not in unique_exception_player:
        print 'Processing: ', name
        TS = group.iloc[::-1]
        rng = pd.date_range('1/1/2000', periods = len(TS), freq = 'D')
        TS['Date_New'] = rng
        TS = TS.set_index('Date_New')
        TS = TS['DKP']
        #build the regression model
        try:
            model = AR(TS)
            model_fit = model.fit()
            predicted_value = model_fit.predict(start = len(TS) - 1, end = len(TS) - 1, dynamic = False)
        except Exception as e:
            print e.args
            TS = TS.fillna(method = 'ffill')
            model = AR(TS)
            model_fit = model.fit()
            predicted_value = model_fit.predict(start = len(TS) - 1, end = len(TS) - 1, dynamic = False)
        predicted = pd.DataFrame(predicted_value)
        predicted = predicted.reset_index()
        predicted.columns = ['Date', 'Predictions']
        del predicted['Date']
        predicted['Player'] = name
        AR_predictions = pd.concat([AR_predictions, predicted], axis = 0)

Processing:  AJ Hammons
Processing:  Aaron Brooks
Processing:  Aaron Gordon
Processing:  Aaron Harrison
Processing:  Adreian Payne


/Users/royh21k/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Processing:  Al Horford
Processing:  Al-Farouq Aminu
Processing:  Alan Anderson
Processing:  Alan Williams
Processing:  Alex Abrines
Processing:  Alex Len
Processing:  Alexis Ajinca
Processing:  Allen Crabbe
Processing:  Alonzo Gee
Processing:  Amir Johnson
Processing:  Anderson Varejao
Processing:  Andre Drummond
Processing:  Andre Iguodala
Processing:  Andre Roberson
Processing:  Andrew Bogut
Processing:  Andrew Harrison
Processing:  Andrew Nicholson
Processing:  Andrew Wiggins
Processing:  Anthony Bennett
Processing:  Anthony Davis
Processing:  Anthony Morrow
Processing:  Anthony Tolliver
Processing:  Arinze Onuaku
Processing:  Aron Baynes
Processing:  Austin Rivers
Processing:  Avery Bradley
Processing:  Ben McLemore
Processing:  Beno Udrih
Processing:  Bismack Biyombo
Processing:  Blake Griffin
Processing:  Boban Marjanovic
Processing:  Bobby Brown
Processing:  Bobby Portis
Processing:  Bojan Bogdanovic
Processing:  Boris Diaw
Processing:  Bradley Beal
Processing:  Brandan Wright


In [14]:
#Check prediction results
AR_predictions.head()

Predictions          Player
0     0.099762      AJ Hammons
0     7.475522    Aaron Brooks
0    16.967870    Aaron Gordon
0     0.592032  Aaron Harrison
0     0.401777   Adreian Payne

In [15]:
len(AR_predictions)

388

In [16]:
#Get actual DKP's
roto1 = roto1.dropna()
#exception_player = []
#AR_predictions = pd.DataFrame([])
actual_DK_pos = []
for name,group in roto1.groupby('First  Last'):
    if name not in unique_exception_player:
        print 'Processing: ', name
        TS = group.iloc[::-1]
        rng = pd.date_range('1/1/2000', periods = len(TS), freq = 'D')
        TS['Date_New'] = rng
        TS = TS.set_index('Date_New')
        TS = TS['DKP']
        #build the regression model
        actual_DK_pos.append(TS[-1])

Processing:  AJ Hammons
Processing:  Aaron Brooks
Processing:  Aaron Gordon
Processing:  Aaron Harrison
Processing:  Adreian Payne
Processing:  Al Horford


/Users/royh21k/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Processing:  Al-Farouq Aminu
Processing:  Alan Anderson
Processing:  Alan Williams
Processing:  Alex Abrines
Processing:  Alex Len
Processing:  Alexis Ajinca
Processing:  Allen Crabbe
Processing:  Alonzo Gee
Processing:  Amir Johnson
Processing:  Anderson Varejao
Processing:  Andre Drummond
Processing:  Andre Iguodala
Processing:  Andre Roberson
Processing:  Andrew Bogut
Processing:  Andrew Harrison
Processing:  Andrew Nicholson
Processing:  Andrew Wiggins
Processing:  Anthony Bennett
Processing:  Anthony Davis
Processing:  Anthony Morrow
Processing:  Anthony Tolliver
Processing:  Arinze Onuaku
Processing:  Aron Baynes
Processing:  Austin Rivers
Processing:  Avery Bradley
Processing:  Ben McLemore
Processing:  Beno Udrih
Processing:  Bismack Biyombo
Processing:  Blake Griffin
Processing:  Boban Marjanovic
Processing:  Bobby Brown
Processing:  Bobby Portis
Processing:  Bojan Bogdanovic
Processing:  Boris Diaw
Processing:  Bradley Beal
Processing:  Brandan Wright
Processing:  Brandon Bas

In [17]:
len(actual_DK_pos)

388

In [18]:
if len(AR_predictions) == len(actual_DK_pos):
    print 'Looks good.'

Looks good.


In [19]:
AR_predictions['Actual'] = actual_DK_pos

In [20]:
AR_predictions.head(10)

Predictions           Player  Actual
0     0.099762       AJ Hammons    0.00
0     7.475522     Aaron Brooks   14.00
0    16.967870     Aaron Gordon   21.25
0     0.592032   Aaron Harrison    0.00
0     0.401777    Adreian Payne    6.75
0    30.945297       Al Horford   35.00
0    15.732958  Al-Farouq Aminu   29.50
0     1.562162    Alan Anderson    0.00
0    26.509307    Alan Williams   32.75
0     7.371297     Alex Abrines    2.00

In [21]:
diff = AR_predictions['Predictions'] - AR_predictions['Actual']
diff_sqr = [x**2 for x in diff]
diff_sqr_sum = np.sum(diff_sqr)
MSE = diff_sqr_sum / len(AR_predictions)
np.sqrt(MSE)

7.4112190929282837

# Optimization

In [22]:
toyset = roto1[roto1.Date == pd.to_datetime('2017-03-26')][['First  Last', 'DK Sal', 'DK pos', 'DKP']]
toyset['DK pos'] = toyset['DK pos'].astype(int).astype(str).apply(lambda x: list(x))
dums = toyset['DK pos'].str.join('@').str.get_dummies('@')
dums.rename(columns={'1':'pg','2':'sg','3':'sf','4':'pf','5':'c'}, inplace=True)
dums = dums.reset_index()
del dums['index']
dums.head(5)

pg  sg  sf  pf  c
0   1   1   0   0  0
1   1   0   0   0  0
2   0   1   0   0  0
3   0   0   0   0  1
4   0   0   0   0  1

In [23]:
toyset = toyset.reset_index()
del toyset['index']
toyset = pd.concat([toyset, dums], axis = 1 )

In [24]:
toyset.head()

First  Last  DK Sal  DK pos    DKP  pg  sg  sf  pf  c
0       Lou Williams  4800.0  [1, 2]  44.75   1   1   0   0  0
1         Quinn Cook  3000.0     [1]   8.25   1   0   0   0  0
2   Tim Hardaway Jr.  6300.0     [2]  28.00   0   1   0   0  0
3        Robin Lopez  4800.0     [5]  29.25   0   0   0   0  1
4  Cristiano Felicio  3000.0     [5]   0.00   0   0   0   0  1

In [25]:
AR_prediction_result = AR_predictions.ix[:, AR_predictions.columns != 'Actual']
AR_prediction_result.head()

Predictions          Player
0     0.099762      AJ Hammons
0     7.475522    Aaron Brooks
0    16.967870    Aaron Gordon
0     0.592032  Aaron Harrison
0     0.401777   Adreian Payne

In [26]:
toyset = toyset.merge(AR_prediction_result, left_on = 'First  Last', right_on = 'Player')
del toyset['Player']

In [27]:
toyset.head()

First  Last   DK Sal  DK pos    DKP  pg  sg  sf  pf  c  Predictions
0      Lou Williams   4800.0  [1, 2]  44.75   1   1   0   0  0    30.180650
1  Tim Hardaway Jr.   6300.0     [2]  28.00   0   1   0   0  0    29.843004
2       Robin Lopez   4800.0     [5]  29.25   0   0   0   0  1    27.262215
3      Vince Carter   3900.0     [3]  17.75   0   0   1   0  0    12.806367
4      James Harden  12500.0  [1, 2]  46.25   1   1   0   0  0    58.474437

In [28]:
DK_pos = toyset['DK pos'].tolist()
pos_string = [['PG' if x==str(1) else('SG' if x==str(2) else('SF' if x==str(3) else('PF' if x==str(4) else('C')))) for x in y] for y in DK_pos] 
print DK_pos[:7]
print pos_string[:7]

[['1', '2'], ['2'], ['5'], ['3'], ['1', '2'], ['1', '2'], ['1']]
[['PG', 'SG'], ['SG'], ['C'], ['SF'], ['PG', 'SG'], ['PG', 'SG'], ['PG']]


In [29]:
toyset1 = toyset.copy()
toyset1['pos_string'] = pos_string
del toyset1['DK pos']
toyset1.head(7)

First  Last   DK Sal    DKP  pg  sg  sf  pf  c  Predictions pos_string
0      Lou Williams   4800.0  44.75   1   1   0   0  0    30.180650   [PG, SG]
1  Tim Hardaway Jr.   6300.0  28.00   0   1   0   0  0    29.843004       [SG]
2       Robin Lopez   4800.0  29.25   0   0   0   0  1    27.262215        [C]
3      Vince Carter   3900.0  17.75   0   0   1   0  0    12.806367       [SF]
4      James Harden  12500.0  46.25   1   1   0   0  0    58.474437   [PG, SG]
5      Jrue Holiday   7200.0  34.50   1   1   0   0  0    32.486918   [PG, SG]
6       Tim Frazier   3900.0  27.50   1   0   0   0  0    20.355008       [PG]

In [30]:
class Player():
    def __init__(self, position, name, salary, points, value):#, pg, sg, sf, pf, c):
        self.self = self
        self.position = position
        self.name = name
        self.salary = salary
        self.points = points
        self.value = value

    def __iter__(self):
        return iter(self.list)
    
    def __str__(self):
        return "{} {} {} {}".format(self.name,self.position,self.salary, self.points)

In [31]:
players = []
for i in range(len(toyset1)):
    name = toyset1.iloc[i, 0]
    position = toyset1.iloc[i, -1]
    salary = toyset1.iloc[i, 1]
    points = toyset1.iloc[i, 8]
    value = points / salary
    #pg = toyset.iloc[i, -5]
    #sg = toyset.iloc[i, -4]
    #sf = toyset.iloc[i, -3]
    #pf = toyset.iloc[i, -2]
    #c = toyset.iloc[i, -1]
    player = Player(position, name, salary, points, value)#, pg, sg, sf, pf, c)
    players.append(player)

In [32]:
def pos_check_fun(pos, constraints, counts):
    n = 0
    for i in counts.keys():
        n += counts[i]
    if n == len(counts.keys()):
        return False
    if counts['UTIL'] == 0:
        return True
    for entry in pos:
        if 'G' in entry:
            if counts['G'] == 0:
                return True
        elif 'F' in entry:
            if counts['F'] == 0:
                return True
    result_lis = []
    for entry in pos:
        result_lis.append(counts[entry] < constraints[entry])
    result_array = np.array(result_lis)
    return result_array.all()

In [33]:
def improved_knapsack(players):
    budget = 50000
    current_team_salary = 0
    constraints = {
        'PG':1,
        'SG':1,
        'SF':1,
        'PF':1,
        'C': 1,
        'G': 1,
        'F': 1,
        'UTIL':1
        }
    
    counts = {
        'PG':0,
        'SG':0,
        'SF':0,
        'PF':0,
        'C': 0,
        'G': 0,
        'F': 0,
        'UTIL':0
        }
    
    players.sort(key=lambda x: x.value, reverse=True)
    team = []
    
    G = 0
    F = 0
    for player in players:
        nam = player.name
        pos = player.position
        sal = player.salary
        pts = player.points
        if pos_check_fun(pos, constraints, counts) and current_team_salary + sal <= budget:
            team.append(player)
            if counts['PG'] == 0 and 'PG' in pos:
                counts['PG'] += 1
            elif counts['SG'] == 0 and 'SG' in pos:
                counts['SG'] += 1
            elif 'PG' in pos and counts['PG'] != 0 and G == 0:
                counts['G'] += 1
            elif 'SG' in pos and counts['SG'] != 0 and G == 0:
                counts['G'] += 1 
            elif counts['PF'] == 0 and 'PF' in pos:
                counts['PF'] += 1
            elif counts['SF'] == 0 and 'SF' in pos:
                counts['SF'] += 1
            elif 'PF' in pos and counts['PF'] != 0 and F == 0:
                counts['F'] += 1
            elif 'SF' in pos and counts['SF'] != 0 and F == 0:
                counts['F'] += 1
            elif counts['C'] == 0 and 'C' in pos:
                counts['C'] += 1
            else:
                counts['UTIL'] +=1
            current_team_salary += sal
    
    players.sort(key=lambda x: x.points, reverse=True)
    for player in players:
        nam = player.name
        pos = player.position
        sal = player.salary
        pts = player.points
        if player not in team:
            pos_players = [ x for x in team if x.position == pos]
            pos_players.sort(key=lambda x: x.points)
            for pos_player in pos_players:
                if (current_team_salary + sal - pos_player.salary) <= budget and pts > pos_player.points:
                    team[team.index(pos_player)] = player
                    current_team_salary = current_team_salary + sal - pos_player.salary
                    break
    return team

In [34]:
team = improved_knapsack(players)
points = 0
salary = 0
print '****************'
for player in team:
    points += player.points
    salary += player.salary
    print player
print "\nPoints: {}".format(points)
print "Salary: {}".format(salary)

****************
DeAndre Jordan ['C'] 6600.0 49.1915210408
Cody Zeller ['C'] 5100.0 33.8054356062
Devin Booker ['SG'] 7700.0 45.3338085199
KJ McDaniels ['SF'] 3200.0 21.3424328099
David West ['PF'] 3000.0 19.480963299
Klay Thompson ['SG'] 7200.0 41.5240398386
Amir Johnson ['PF'] 3900.0 24.427817089
Russell Westbrook ['PG'] 13000.0 67.5497011998

Points: 302.655719403
Salary: 49700.0


In [35]:
l = []
for i in range(len(team)):
    l.append(team[i].name)

In [36]:
toyset[toyset['First  Last'].isin(l)]

First  Last   DK Sal DK pos    DKP  pg  sg  sf  pf  c  Predictions
33      DeAndre Jordan   6600.0    [5]  39.25   0   0   0   0  1    49.191521
72   Russell Westbrook  13000.0    [1]  75.75   1   0   0   0  0    67.549701
113        Cody Zeller   5100.0    [5]  35.25   0   0   0   0  1    33.805436
119         David West   3000.0    [4]  11.75   0   0   0   1  0    19.480963
128       Amir Johnson   3900.0    [4]  34.25   0   0   0   1  0    24.427817
143       KJ McDaniels   3200.0    [3]  19.00   0   0   1   0  0    21.342433
152       Devin Booker   7700.0    [2]  36.50   0   1   0   0  0    45.333809
188      Klay Thompson   7200.0    [2]  38.00   0   1   0   0  0    41.524040

# Weight-Adjusted Time Series Regressions

In [37]:
#Regression loops
roto1 = roto1.dropna()
#exception_player = []
EWMA_predictions = pd.DataFrame([])
for name,group in roto1.groupby('First  Last'):
    if name not in unique_exception_player:
        print 'Processing: ', name
        TS = group.iloc[::-1]
        rng = pd.date_range('1/1/2000', periods = len(TS), freq = 'D')
        TS['Date_New'] = rng
        TS = TS.set_index('Date_New')
        TS = TS['DKP']
        TS = pd.ewma(TS.values, alpha = 0.97)
        #build the regression model
        try:
            model = AR(TS)
            model_fit = model.fit()
            predicted_value = model_fit.predict(start = len(TS) - 1, end = len(TS) - 1, dynamic = False)
        except Exception as e:
            print e.args
            TS = TS.fillna(method = 'ffill')
            model = AR(TS)
            model_fit = model.fit()
            predicted_value = model_fit.predict(start = len(TS) - 1, end = len(TS) - 1, dynamic = False)
        predicted = pd.DataFrame(predicted_value)
        predicted = predicted.reset_index()
        predicted.columns = ['Date', 'Predictions']
        del predicted['Date']
        predicted['Player'] = name
        EWMA_predictions = pd.concat([EWMA_predictions, predicted], axis = 0)

Processing:  AJ Hammons
Processing:  Aaron Brooks
Processing:  Aaron Gordon
Processing:  Aaron Harrison
Processing:  Adreian Payne
Processing:  Al Horford
Processing:  Al-Farouq Aminu


/Users/royh21k/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/royh21k/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:13: FutureWarning: pd.ewm_mean is deprecated for ndarrays and will be removed in a future version


Processing:  Alan Anderson
Processing:  Alan Williams
Processing:  Alex Abrines
Processing:  Alex Len
Processing:  Alexis Ajinca
Processing:  Allen Crabbe
Processing:  Alonzo Gee
Processing:  Amir Johnson
Processing:  Anderson Varejao
Processing:  Andre Drummond
Processing:  Andre Iguodala
Processing:  Andre Roberson
Processing:  Andrew Bogut
Processing:  Andrew Harrison
Processing:  Andrew Nicholson
Processing:  Andrew Wiggins
Processing:  Anthony Bennett
Processing:  Anthony Davis
Processing:  Anthony Morrow
Processing:  Anthony Tolliver
Processing:  Arinze Onuaku
Processing:  Aron Baynes
Processing:  Austin Rivers
Processing:  Avery Bradley
Processing:  Ben McLemore
Processing:  Beno Udrih
Processing:  Bismack Biyombo
Processing:  Blake Griffin
Processing:  Boban Marjanovic
Processing:  Bobby Brown
Processing:  Bobby Portis
Processing:  Bojan Bogdanovic
Processing:  Boris Diaw
Processing:  Bradley Beal
Processing:  Brandan Wright
Processing:  Brandon Bass
Processing:  Brandon Jennin

In [38]:
EWMA_predictions.head(10)

Predictions           Player
0     0.096120       AJ Hammons
0     7.656028     Aaron Brooks
0    17.136273     Aaron Gordon
0     0.653905   Aaron Harrison
0     0.388083    Adreian Payne
0    31.239458       Al Horford
0    15.791603  Al-Farouq Aminu
0     1.504437    Alan Anderson
0    26.520112    Alan Williams
0     7.731948     Alex Abrines

In [39]:
AR_predictions.head(10)

Predictions           Player  Actual
0     0.099762       AJ Hammons    0.00
0     7.475522     Aaron Brooks   14.00
0    16.967870     Aaron Gordon   21.25
0     0.592032   Aaron Harrison    0.00
0     0.401777    Adreian Payne    6.75
0    30.945297       Al Horford   35.00
0    15.732958  Al-Farouq Aminu   29.50
0     1.562162    Alan Anderson    0.00
0    26.509307    Alan Williams   32.75
0     7.371297     Alex Abrines    2.00

In [40]:
combined = pd.concat([AR_predictions, EWMA_predictions], axis = 1)
combined.columns = ('AR Predictions', 'Player1', 'Actual', 'EWMA Predictions', 'Player')
del combined['Player1']
combined.head(10)

AR Predictions  Actual  EWMA Predictions           Player
0        0.099762    0.00          0.096120       AJ Hammons
0        7.475522   14.00          7.656028     Aaron Brooks
0       16.967870   21.25         17.136273     Aaron Gordon
0        0.592032    0.00          0.653905   Aaron Harrison
0        0.401777    6.75          0.388083    Adreian Payne
0       30.945297   35.00         31.239458       Al Horford
0       15.732958   29.50         15.791603  Al-Farouq Aminu
0        1.562162    0.00          1.504437    Alan Anderson
0       26.509307   32.75         26.520112    Alan Williams
0        7.371297    2.00          7.731948     Alex Abrines

In [41]:
#Get the last game dates for all players for whom we made predictions
last_game_dates = []
for name,group in roto1.groupby('First  Last'):
    if name not in unique_exception_player:
        print 'Processing: ', name
        TSA = group.iloc[::-1]
        last_game_date = TSA.Date.values[0]
        last_game_dates.append(last_game_date)

Processing:  AJ Hammons
Processing:  Aaron Brooks
Processing:  Aaron Gordon
Processing:  Aaron Harrison
Processing:  Adreian Payne
Processing:  Al Horford
Processing:  Al-Farouq Aminu
Processing:  Alan Anderson
Processing:  Alan Williams
Processing:  Alex Abrines
Processing:  Alex Len
Processing:  Alexis Ajinca
Processing:  Allen Crabbe
Processing:  Alonzo Gee
Processing:  Amir Johnson
Processing:  Anderson Varejao
Processing:  Andre Drummond
Processing:  Andre Iguodala
Processing:  Andre Roberson
Processing:  Andrew Bogut
Processing:  Andrew Harrison
Processing:  Andrew Nicholson
Processing:  Andrew Wiggins
Processing:  Anthony Bennett
Processing:  Anthony Davis
Processing:  Anthony Morrow
Processing:  Anthony Tolliver
Processing:  Arinze Onuaku
Processing:  Aron Baynes
Processing:  Austin Rivers
Processing:  Avery Bradley
Processing:  Ben McLemore
Processing:  Beno Udrih
Processing:  Bismack Biyombo
Processing:  Blake Griffin
Processing:  Boban Marjanovic
Processing:  Bobby Brown
Pro

In [42]:
#Add the last game dates to the info matrix
#combined['Last Game Date'] = last_game_dates#Commented because we only need to run it once.
combined.head()

AR Predictions  Actual  EWMA Predictions          Player
0        0.099762    0.00          0.096120      AJ Hammons
0        7.475522   14.00          7.656028    Aaron Brooks
0       16.967870   21.25         17.136273    Aaron Gordon
0        0.592032    0.00          0.653905  Aaron Harrison
0        0.401777    6.75          0.388083   Adreian Payne

In [43]:
combined.to_csv('GamePredictions.csv', index = False)

# Optimization on EWMA results

In [44]:
toyset = roto1[roto1.Date == pd.to_datetime('2017-03-26')][['First  Last', 'DK Sal', 'DK pos', 'DKP']]
toyset['DK pos'] = toyset['DK pos'].astype(int).astype(str).apply(lambda x: list(x))
dums = toyset['DK pos'].str.join('@').str.get_dummies('@')
dums.rename(columns={'1':'pg','2':'sg','3':'sf','4':'pf','5':'c'}, inplace=True)
dums = dums.reset_index()
del dums['index']
dums.head(5)

pg  sg  sf  pf  c
0   1   1   0   0  0
1   1   0   0   0  0
2   0   1   0   0  0
3   0   0   0   0  1
4   0   0   0   0  1

In [45]:
toyset = toyset.reset_index()
del toyset['index']
toyset = pd.concat([toyset, dums], axis = 1 )

In [46]:
toyset.head()

First  Last  DK Sal  DK pos    DKP  pg  sg  sf  pf  c
0       Lou Williams  4800.0  [1, 2]  44.75   1   1   0   0  0
1         Quinn Cook  3000.0     [1]   8.25   1   0   0   0  0
2   Tim Hardaway Jr.  6300.0     [2]  28.00   0   1   0   0  0
3        Robin Lopez  4800.0     [5]  29.25   0   0   0   0  1
4  Cristiano Felicio  3000.0     [5]   0.00   0   0   0   0  1

In [47]:
EWMA_predictions.head()

Predictions          Player
0     0.096120      AJ Hammons
0     7.656028    Aaron Brooks
0    17.136273    Aaron Gordon
0     0.653905  Aaron Harrison
0     0.388083   Adreian Payne

In [48]:
toyset = toyset.merge(EWMA_predictions, left_on = 'First  Last', right_on = 'Player')
del toyset['Player']

In [49]:
toyset.head()

First  Last   DK Sal  DK pos    DKP  pg  sg  sf  pf  c  Predictions
0      Lou Williams   4800.0  [1, 2]  44.75   1   1   0   0  0    29.536480
1  Tim Hardaway Jr.   6300.0     [2]  28.00   0   1   0   0  0    30.058803
2       Robin Lopez   4800.0     [5]  29.25   0   0   0   0  1    26.570875
3      Vince Carter   3900.0     [3]  17.75   0   0   1   0  0    12.887248
4      James Harden  12500.0  [1, 2]  46.25   1   1   0   0  0    59.295521

In [50]:
DK_pos = toyset['DK pos'].tolist()
pos_string = [['PG' if x==str(1) else('SG' if x==str(2) else('SF' if x==str(3) else('PF' if x==str(4) else('C')))) for x in y] for y in DK_pos] 
print DK_pos[:7]
print pos_string[:7]

[['1', '2'], ['2'], ['5'], ['3'], ['1', '2'], ['1', '2'], ['1']]
[['PG', 'SG'], ['SG'], ['C'], ['SF'], ['PG', 'SG'], ['PG', 'SG'], ['PG']]


In [51]:
toyset1 = toyset.copy()
toyset1['pos_string'] = pos_string
del toyset1['DK pos']
toyset1.head(7)

First  Last   DK Sal    DKP  pg  sg  sf  pf  c  Predictions pos_string
0      Lou Williams   4800.0  44.75   1   1   0   0  0    29.536480   [PG, SG]
1  Tim Hardaway Jr.   6300.0  28.00   0   1   0   0  0    30.058803       [SG]
2       Robin Lopez   4800.0  29.25   0   0   0   0  1    26.570875        [C]
3      Vince Carter   3900.0  17.75   0   0   1   0  0    12.887248       [SF]
4      James Harden  12500.0  46.25   1   1   0   0  0    59.295521   [PG, SG]
5      Jrue Holiday   7200.0  34.50   1   1   0   0  0    32.721360   [PG, SG]
6       Tim Frazier   3900.0  27.50   1   0   0   0  0    20.566667       [PG]

In [52]:
players = []
for i in range(len(toyset1)):
    name = toyset1.iloc[i, 0]
    position = toyset1.iloc[i, -1]
    salary = toyset1.iloc[i, 1]
    points = toyset1.iloc[i, 8]
    value = points / salary
    #pg = toyset.iloc[i, -5]
    #sg = toyset.iloc[i, -4]
    #sf = toyset.iloc[i, -3]
    #pf = toyset.iloc[i, -2]
    #c = toyset.iloc[i, -1]
    player = Player(position, name, salary, points, value)#, pg, sg, sf, pf, c)
    players.append(player)

In [53]:
team = improved_knapsack(players)
points = 0
salary = 0
print '****************'
for player in team:
    points += player.points
    salary += player.salary
    print player
print "\nPoints: {}".format(points)
print "Salary: {}".format(salary)

****************
DeAndre Jordan ['C'] 6600.0 48.6038117076
Dwight Howard ['C'] 7600.0 41.1974237687
Devin Booker ['SG'] 7700.0 46.7204956086
KJ McDaniels ['SF'] 3200.0 21.2438160392
David West ['PF'] 3000.0 19.6444063977
Amir Johnson ['PF'] 3900.0 24.6183096967
Andre Iguodala ['SG'] 4500.0 27.9978430461
Russell Westbrook ['PG'] 13000.0 67.6011902925

Points: 297.627296557
Salary: 49500.0


## The result is better than before (Points: 293.270792979).